In [215]:
import eval7

import random


Card = eval7.Card
Hand = tuple[Card, ...]


def hs(hand: Hand, maxiter: int) -> float:
    cards = [*eval7.Deck().cards]
    for card in hand:
        cards.remove(card)

    sco = 0
    for _ in range(maxiter):
        random.shuffle(cards)

        if len(hand) < 7 or hand[-1].suit in (1, 2):
            i = 0
            while i + len(hand) < 7 or cards[i].suit in (1, 2):
                i += 1
            community = cards[:i + 1]
            othe = tuple(cards[i + 1:i + 3])
        else:
            community = cards[:0]
            othe = tuple(cards[:2])
        
        us = eval7.evaluate(hand + tuple(community))
        op = eval7.evaluate(othe + hand[2:] + tuple(community))
        sco += 2 if us > op else 1 if us == op else 0

    return sco / (2 * maxiter)

In [216]:
hs((Card('Ad'), Card('Ah'), Card('Ac')), 100)

0.93

In [289]:
from itertools import combinations


def profile(hand: tuple[Card, ...], maxiter: int) -> list[float]:
    cards = [*eval7.Deck().cards]
    for card in hand:
        cards.remove(card)

    vals = []
    for _ in range(maxiter):
        random.shuffle(cards)

        if len(hand) != 7 or hand[-1].suit in (1, 2):
            i = 0
            while i + len(hand) < 7 or cards[i].suit in (1, 2):
                i += 1
            community = cards[:i + 1]
        else:
            community = cards[:0]
        
        v = hs(hand + tuple(community), 30)
        vals.append(v + (random.random() - 1/2) * 0.01)
        # vals.append(v)
    
    return vals

In [290]:
from tqdm.notebook import tqdm

In [291]:
profiles = {}
for hand in tqdm(combinations(eval7.Deck().cards, 2)):
    profiles[hand] = profile(hand, 100)

0it [00:00, ?it/s]

In [182]:
def emd(h1: list[float], h2: list[float]) -> float:
    d = 0

    carry = 0
    for v1, v2 in zip(h1, h2):
        d += abs(carry)
        carry = v1 + carry - v2

    return d